In [1]:
#libraries required
import pandas as pd
import mysql.connector
import string
import json
import psycopg2
from jsonpath_ng.ext import parse

In [2]:
#Functions
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#library that contains punctuation
#defining the function to remove punctuation
def remove_punctuation_and_lower(text):
    punctuationfree="".join([i.lower() for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text



def load_json_data(input_file):
    try:
        with open(input_file, 'r') as data_json:
            source_data = json.load(data_json)
        return source_data
    except FileNotFoundError:
        print(f"Error: The file '{input_file}' was not found.")
        return None

In [3]:
# database = input("Enter database name:")
# host = input("Enter host name:")
# user = input("Enter user name:")
# password = input("Enter password:")
# port = input("Enter port number:")

conn = psycopg2.connect(database="nourish",
                        host="awesome-hw.sdsc.edu",
                        user="v1desai@ucsd.edu",
                        password="emfQGcx3",
                        port=5432)

cur = conn.cursor()

In [4]:
#product_ingredients_2018

list_of_tables = ['usda_2022_food_branded_experimental', 'usda_2022_branded_food_nutrients', 'usda_2022_nutrient_master']

cur.execute("""SELECT t1.fdc_id, t1.description, t2.nutrient_id, t2.amount, t3.name, t3.unit_name, t3.nutrient_nbr, t3.rank
            FROM usda_2022_food_branded_experimental 
            t1 LEFT JOIN usda_2022_branded_food_nutrients t2 ON t1.fdc_id = t2.fdc_id LEFT JOIN
            usda_2022_nutrient_master t3 ON t2.nutrient_id = t3.id LIMIT 100""")
join_records = cur.fetchall()
columns = [desc[0] for desc in cur.description]
join_df = pd.DataFrame(join_records, columns=columns)
group_object = join_df.groupby(['description'])['nutrient_nbr'].mean().reset_index()

In [5]:
pd.set_option('display.max_colwidth', None)
group_object.description



0    72% COCOA CRANBERRIES ALMONDS + DARK CHOCOLATE, CRANBERRIES ALMONDS
1                                           BEET WHOLE GRAIN CHIPS, BEET
2                                         MINT CHIP ICE CREAM, MINT CHIP
3                                              MUENSTER CHEESE, MUENSTER
4                       RASPBERRY CREAM TOP WHOLE MILK YOGURT, RASPBERRY
5                     STRAWBERRY ORGANIC TWISTED FRUIT SNACK, STRAWBERRY
6                                                  WHOLE PEELED TOMATOES
Name: description, dtype: object

In [6]:
for tbl in list_of_tables:
    table_type_column_query = f"""SELECT column_name, data_type 
                             FROM information_schema.columns
                             WHERE table_name = '{tbl}';"""
    cur.execute(table_type_column_query)
    records = cur.fetchall()
    print('\n')
    print(tbl)
    print(pd.DataFrame(records))




usda_2022_food_branded_experimental
                  0        1
0            fdc_id  integer
1         data_type     text
2       description     text
3  food_category_id     text
4  publication_date     text


usda_2022_branded_food_nutrients
                    0        1
0                  id  integer
1              fdc_id  integer
2         nutrient_id  integer
3              amount  numeric
4         data_points     text
5       derivation_id  integer
6                 min     text
7                 max     text
8              median     text
9            footnote     text
10  min_year_acquired     text


usda_2022_nutrient_master
              0        1
0            id  integer
1          name     text
2     unit_name     text
3  nutrient_nbr  numeric
4          rank     text


In [2]:
#prof deleted this table -- probably not good
#foods_to_avoid = pd.read_csv('Disease_foods_to_avoid_csv.csv', delimiter = ';')
#load in tasty dishes (csv)
tasty_dishes = pd.read_csv('dishes.csv') 
#load in tasty ingredients and instructions as Pandas dataframe
with open('ingredient_and_instructions.json') as file:
    tasty_json_ingredients = json.load(file)
ingredients_and_recipes_df = pd.DataFrame(tasty_json_ingredients)


In [3]:
#unstructured data
jsonpath_expr = parse("$..['instructions']")
matches = [match.value for match in jsonpath_expr.find(tasty_json_ingredients)]

text = []
for i in range(len(matches)):
    list_text = " ".join(line['display_text'].strip() for line in matches[i])
    text.append(list_text)

In [17]:
jsonpath_expr = parse("$..['ingredient_sections']")
matches = [match.value for match in jsonpath_expr.find(tasty_json_ingredients)]

In [18]:
#extracting list of ingredients product by product
ingredients = []
ingredient_per_prod = []
for products in matches:
    for product in products:
        ing_per_prod = product['ingredients']
        for ing in ing_per_prod:
            ingredient_per_prod.append(ing['name'])
    ingredients.append(ingredient_per_prod)
    ingredient_per_prod = []

In [19]:
metric = []
metric_per_prod = []
for products in matches:
    for product in products:
        met_per_prod = product['ingredients']
        for met in met_per_prod:
            if met['metric_unit'] is not None:
                metric_per_prod.append((met['metric_unit']['quantity'],met['metric_unit']['display']))
            else:
                metric_per_prod.append((None,None))
    metric.append(metric_per_prod)
    metric_per_prod = []

In [72]:
tasty_dishes_filtered = tasty_dishes[['keywords', 'name', 'protein', 'fat', 'calories', 'sugar', 'carbohydrates', 'fiber']]